In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import mne
import os
import re
import torch
import pickle

from utils import *

In [2]:
task_dict = {
    'resting_state': {'annotation': 'T0', 'runs': ['03', '04', '05', '06','07', '08', '09', '10', '11', '12', '13', '14']},
    'left_fist_performed': {'annotation': 'T1', 'runs': ['03', '07' '11']},
    'left_fist_imagined': {'annotation': 'T1', 'runs': ['04', '08', '12']},
    'both_fists_performed': {'annotation': 'T1', 'runs': ['05', '09', '13']},
    'both_fists_imagined': {'annotation': 'T1', 'runs': ['06', '10', '14']},
    'right_fist_performed': {'annotation': 'T2', 'runs': ['03', '07', '11']},
    'right_fist_imagined': {'annotation': 'T2', 'runs': ['04', '08', '12']},
    'both_feet_performed': {'annotation': 'T2', 'runs': ['05', '09', '13']},
    'both_feet_imagined': {'annotation': 'T2', 'runs': ['06', '10', '14']}
}


label_dict = {
    'T0': {'03': 'Resting state', '04': 'Resting state', '05': 'Resting state', '06': 'Resting state', '07': 'Resting state', '08': 'Resting state', '09': 'Resting state', '10': 'Resting state', '11': 'Resting state', '12': 'Resting state', '13': 'Resting state', '14': 'Resting state'},
    'T1': {'03': 'Left fist, performed', '04': 'Left fist, imagined', '05': 'Both fists, performed', '06': 'Both fists, imagined', '07': 'Left fist, performed', '08': 'Left fist, imagined', '09': 'Both fists, performed', '10': 'Both fists, imagined', '11': 'Left fist, performed', '12': 'Left fist, imagined', '13': 'Both fists, performed', '14': 'Both fists, imagined'},
    'T2': {'03': 'Right fist, performed', '04': 'Right fist, imagined', '05': 'Both feet, performed', '06': 'Both feet, imagined', '07': 'Right fist, performed', '08': 'Right fist, imagined', '09': 'Both feet, performed', '10': 'Both feet, imagined', '11': 'Right fist, performed', '12': 'Right fist, imagined', '13': 'Both feet, performed', '14': 'Both feet, imagined'}
}

In [3]:
DATA_PATH_RAW = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (raw)/files/'
DATA_PATH_LABELLED = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (labelled)/'


# create folder for each label
for label in task_dict.keys():
    if not os.path.exists(DATA_PATH_LABELLED + label):
        os.makedirs(DATA_PATH_LABELLED + label)

In [4]:
NUMBER_SAMPLES = 1024
NUMBER_CHANNELS = 20

for patient_folder in tqdm(os.listdir(DATA_PATH_RAW)):
    for file in os.listdir(DATA_PATH_RAW + patient_folder + '/'):
        if file.endswith('.edf') and file[5:7] not in ['01', '02']:
            file_path = DATA_PATH_RAW + patient_folder + '/' + file

            raw = get_raw(file_path)
            annotations = get_annotations(file_path)

            annotation_dict = get_window_dict(raw, annotations)

            for annotation in annotation_dict.keys():
                for raw in annotation_dict[annotation]:
                    raw = pick_and_rename_channels(raw.copy())

                    

                    try: 
                        x = np.zeros((1, NUMBER_CHANNELS, NUMBER_SAMPLES))
                        x[:,:19,:] = raw.copy().get_data()[:,:NUMBER_SAMPLES].reshape(1,NUMBER_CHANNELS-1,NUMBER_SAMPLES)
                        x[:,19,:] = np.ones((1, NUMBER_SAMPLES)) * -1  
                        x = torch.from_numpy(x).float()

                        label = label_dict[annotation][file[5:7]]
                        
                        picklePath = DATA_PATH_LABELLED + label + '/' + file[0:7] + '_' + annotation + '_' + label + '.pkl'
                        with open(picklePath, 'wb') as handle:
                            pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)
                    except:
                        print('Error with file: ' + file, x.shape)
                    

  0%|          | 0/109 [00:00<?, ?it/s]

Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1050)
Error with file: S048R05.edf (19, 1051)
Error with file: S048R05.edf (19, 1051)


  1%|          | 1/109 [00:06<11:15,  6.25s/it]

Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1051)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1050)
Error with file: S048R10.edf (19, 1051)


  2%|▏         | 2/109 [00:12<10:55,  6.12s/it]

...

  3%|▎         | 3/109 [00:18<10:35,  6.00s/it]

Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1050)
Error with file: S059R06.edf (19, 1051)


  3%|▎         | 3/109 [00:22<13:21,  7.56s/it]

Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)
Error with file: S009R14.edf (19, 1051)
Error with file: S009R14.edf (19, 1050)


KeyboardInterrupt: 

In [ ]:
raw.info['ch_names']